In [1]:
# some general packages
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # suppress all that TF output

# ML packages
import tensorflow as tf
tfk = tf.keras
from tensorflow.keras import initializers
import cymetric

# sage integration
from cymetric.sage.sagelib import prepare_toric_cy_data

# data generation
from cymetric.pointgen.pointgen_mathematica import PointGeneratorMathematica, PointGeneratorToricMathematica
from cymetric.pointgen.pointgen_toric import ToricPointGenerator
from cymetric.models.tfhelper import prepare_tf_basis
from cymetric.pointgen.nphelper import prepare_dataset, prepare_basis_pickle

# training
from cymetric.models.tfmodels import PhiFSModel, PhiFSModelToric
from cymetric.models.tfhelper import prepare_tf_basis
from cymetric.models.callbacks import RicciCallback, SigmaCallback, KaehlerCallback, AlphaCallback, VolkCallback, TransitionCallback
from cymetric.models.losses import sigma_loss
from cymetric.models.metrics import SigmaLoss, KaehlerLoss, TransitionLoss, VolkLoss, RicciLoss
from cymetric.models.measures import sigma_measure, transition_measure_loss, ricci_measure

# Example: Random KS model (we just take any with Picard Rank 2)

## Compute toric information

In [3]:
work_dir = "./ToricModel"
vertices = [[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1], [-1, -1, -1, 0], [2, 0, 0, -1]]  # P2 fibered over P2
polytope = LatticePolytope(vertices)
pConfig = PointConfiguration(polytope.points(), star=[0 for _ in range(len(vertices[0]))])
triangs = pConfig.restrict_to_connected_triangulations().restrict_to_fine_triangulations().restrict_to_regular_triangulations().restrict_to_star_triangulations([0 for _ in range(len(vertices[0]))]).triangulations_list()
triang = triangs[0]
tv_fan = triang.fan()
tv = ToricVariety(tv_fan)

toric_data = prepare_toric_cy_data(tv, os.path.join(work_dir, "toric_data.pickle"))
list(toric_data.keys())

['dim_cy',
 'vol_j_norm',
 'coeff_aK',
 'exp_aK',
 'exps_sections',
 'patch_masks',
 'glsm_charges',
 'non_ci_coeffs',
 'non_ci_exps',
 'triple']

## Generate points

In [4]:
num_pts       = int(100000)
precision     = int(10)
verbose       = int(1)
dim_cy        = toric_data['dim_cy']
monomials     = np.array(toric_data['exp_aK'])
coefficients  = np.array(toric_data['coeff_aK'])
sections      = toric_data['exps_sections']
non_ci_coeffs = toric_data['non_ci_coeffs']
non_ci_exps   = toric_data['non_ci_exps']
patch_masks   = toric_data['patch_masks']
glsm_charges  = toric_data['glsm_charges']
kmoduli       = np.ones(len(toric_data['exps_sections']))
dim_ps        = np.array([len(s)+1 for s in toric_data['exps_sections']])

In [5]:
mathematica_pointgen = PointGeneratorToricMathematica(dim_cy, monomials, coefficients, kmoduli, dim_ps, sections, non_ci_coeffs, non_ci_exps, patch_masks, glsm_charges, precision=precision, verbose=verbose)

In [5]:
prepare_dataset(mathematica_pointgen, num_pts, work_dir)
prepare_basis_pickle(mathematica_pointgen, work_dir)

WolframKernel-<tcp://127.0.0.1:42409>:INFO:Connected to logging socket: tcp://127.0.0.1:42409
WolframKernel-<tcp://127.0.0.1:42409>:DEBUG:Evaluating a new expression.
pointgenMathematica:DEBUG:Running with 15 Mathematica kernels.
pointgenMathematica:DEBUG:Initializing generation of 100000 points...
WolframKernel-<tcp://127.0.0.1:42409>:INFO:Number of points on CY from one ambient space intersection: 4
WolframKernel-<tcp://127.0.0.1:42409>:INFO:Now generating 100000 points...
WolframKernel-<tcp://127.0.0.1:42409>:INFO:Generated 0% of points
WolframKernel-<tcp://127.0.0.1:42409>:INFO:Generated 5% of points
WolframKernel-<tcp://127.0.0.1:42409>:INFO:Generated 10% of points
WolframKernel-<tcp://127.0.0.1:42409>:INFO:Generated 15% of points
WolframKernel-<tcp://127.0.0.1:42409>:INFO:Generated 20% of points
WolframKernel-<tcp://127.0.0.1:42409>:INFO:Generated 25% of points
WolframKernel-<tcp://127.0.0.1:42409>:INFO:Generated 30% of points
WolframKernel-<tcp://127.0.0.1:42409>:INFO:Generated 

0

## set up NN

In [6]:
tf.get_logger().setLevel('ERROR')
data = np.load(os.path.join(work_dir, 'dataset.npz'))
BASIS = prepare_tf_basis(np.load(os.path.join(work_dir, 'basis.pickle'), allow_pickle=True))
print([key for key in data])
print([key for key in BASIS])
kappa = 1/np.mean(data['y_train'][:,-2])
nfold = int(BASIS['NFOLD'].numpy().real)
print(1/kappa)

scb = SigmaCallback((data['X_val'], data['y_val']))
kcb = KaehlerCallback((data['X_val'], data['y_val']), data['val_pullbacks'])
tcb = TransitionCallback((data['X_val'], data['y_val']))
rcb = RicciCallback((data['X_val'], data['y_val']), data['val_pullbacks'])
volkck = VolkCallback((data['X_val'], data['y_val']))
cb_list = [scb, volkck]

cmetrics = [SigmaLoss(), VolkLoss()]
loss_fn = sigma_loss(kappa, nfold)


['X_train', 'y_train', 'X_val', 'y_val', 'val_pullbacks']
['DQDZB0', 'DQDZF0', 'QB0', 'QF0', 'NFOLD', 'AMBIENT', 'KMODULI', 'NHYPER']
5.389859450876526


In [7]:
nfold    = int(BASIS['NFOLD'].numpy().real)
amb      = [int(6)]
n_in     = int(12)
n_out    = int(1)
nHiddens = [int(64), int(64), int(64)]
acts     = ['gelu', 'gelu', 'gelu']
nEpochs  = int(50)
bSize    = int(64)
alpha    = [float(1.), float(1.), float(1.), float(1.), float(1.)]

In [8]:
model = tf.keras.Sequential()
model.add(tfk.Input(shape=(int(n_in))))
for nHidden, act in zip(nHiddens, acts):
    model.add(
        tfk.layers.Dense(
            nHidden, 
            activation=act,
            kernel_initializer=initializers.RandomNormal(stddev=float(0.01)),
            # bias_initializer=initializers.Zeros()
        )
    )
model.add(tfk.layers.Dense(n_out))

In [9]:
fs_model = PhiFSModelToric(model, BASIS, alpha=alpha, kappa=kappa, toric_data=toric_data)
fs_model.compile(custom_metrics=cmetrics, optimizer=tfk.optimizers.Adam(), loss=None, run_eagerly=True)  # uses custom loss function in training step, so the argument we pass here is ignored

## train NN

In [10]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'
history = fs_model.fit(data['X_train'], data['y_train'], epochs=nEpochs, batch_size=bSize, verbose=1, callbacks=cb_list)

 - Sigma measure val:      0.4858
 - Volk val:               0.8307
Epoch 1/50


/home/ruehle/.local/lib/python3.9/site-packages/tensorflow/python/framework/indexed_slices.py:448: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradients/PartitionedCall_4_grad/PartitionedCall:1", shape=(None,), dtype=int64), values=Tensor("gradients/PartitionedCall_4_grad/PartitionedCall:0", shape=(None, 6), dtype=complex64), dense_shape=Tensor("gradients/PartitionedCall_4_grad/PartitionedCall:2", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


 358/1407 [======>.......................] - ETA: 20:08 - sigma_loss: 0.4264 - volk_loss: 1.4185e-04

KeyboardInterrupt: 